In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as randint
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
import re # Регулярные выражения.
import string
import datetime
import pymystem3

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold


ohe = OneHotEncoder(sparse = False)
mystem=pymystem3.Mystem()
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

Для вас представлена задача, сделанная на открытых данных.
Вам нужно предсказать цену товара по другим имеющимся характеристикам.
Метрика качества RMSLE.<br>
ЦЕНА в КОПЕЙКАХ!!
Но предсказания нужны в РУБЛЯХ
потому - сразу поделите в train прайс на 100

In [2]:
df_train = pd.read_pickle('./data/train_sample.pckl')

In [ ]:
df_train[:100]

In [ ]:
df_train.info()

In [3]:
def rub(price):
    price = price / 100
    return price

def name_extract (S):
   S = str(S)
   delim = "': '"
   delim2 = "', '"
   return re.split(delim2, re.split(delim, S)[1])[0].strip()

def subcategory_extract (S):
   S = str(S)
   delim = "'name': '"
   delim2 = "', '"
   return re.split(delim2, re.split(delim, S)[2])[0].strip()

def replace_logical(s):
   if s:
       return 1
   else:
       return 0
    
def transformYear(val):
   #year = datetime.datetime.fromtimestamp(int(df_train.date_created[0])).year
   year = datetime.datetime.fromtimestamp(int(val)).year
   return year

def transformMonth(val):
   month = datetime.datetime.fromtimestamp(int(val)).month
   return month

def get_latitude(val):
   res = val['latitude']
   return res

def get_longitude(val):
   res = val['longitude']
   return res

def mystem_func(text):
    return ''.join(mystem.lemmatize(text))

def data_preproc(df_input):
    df_output = df_input.copy() # копируем датафрейм
    df_output = df_output.drop(['category', 'description', 'id', 'images', 'subway', 'subcategory', 'can_buy', 'can_promote'], axis=1) #удаляем ненужные признаки
    df_output.price = df_train['price'].map(rub) # нормируем цену до рублей.
    df_output['category_name'] = df_output['fields'].map(name_extract)
    df_output['subcategory_name'] = df_output['fields'].map(subcategory_extract)
    df_output = df_output.drop(['fields'], axis=1)
    df_output.contacts_visible = df_output.contacts_visible.map(replace_logical)
    df_output.delivery_available = df_output.delivery_available.map(replace_logical)
    df_output.mortgage_available = df_output.mortgage_available.map(replace_logical)
    df_output.payment_available = df_output.payment_available.map(replace_logical)
    df_output['month'] = df_output['date_created'].map(transformMonth)
    df_output['year']=df_output['date_created'].map(transformYear)
    df_output = df_output.drop(['date_created'], axis=1)
    df_output['price'] = df_output['price'].astype(int)
    df_output = df_output[df_output['price']>50]
    df_output['latitude'] = df_output['location'].map(get_latitude)
    df_output['longitude'] = df_output['location'].map(get_longitude)
    df_output = df_output.drop(['location'], axis=1)
    df_output['name'] = df_output['name'].map(mystem_func)
    return df_output
    
    

In [4]:
df_train_preproc = df_train.pipe(data_preproc)

AttributeError: 'Series' object has no attribute 'strip'

In [ ]:
df_train_preproc[:100]

In [ ]:
!telegram-send 'success'